In [1]:
# R code for models for analysis of on-the-fly and CSAI-2 scores as part of
# Analysis III: Self-report Scores

In [3]:
# Load some packages
library(reshape2)
library(ggplot2)
library(cowplot)
library(ez)
library(nlme)
library(multcomp)
library(pastecs)
theme_set(theme_cowplot(font_size=8))
library(usdm)
library(r2glmm)
library(car)
library(MASS)
library(MCMCglmm)
library(lmerTest)
library(expss)
library(moments)

In [5]:
# Read all on-the-fly comfort and anxiety signals among others
signals = read.csv("../metrics/all_events_OnTheFly.csv")
colnames(signals)

[1] "X"            "pid"          "Comfort"      "Anxiety"      "LatDist"     
 [6] "LongDist"     "MaxVel"       "MeanVel"      "MaxLatAcc"    "MeanLatAcc"  
[11] "MaxLongAcc"   "MeanLongAcc"  "MaxLatJerk"   "MeanLatJerk"  "MaxLongJerk" 
[16] "MeanLongJerk" "NumPks"       "MaxPkAmp"     "MeanScl"      "MeanEntropy" 
[21] "MaxEntropy"   "MeanDwell"    "MaxDwell"     "MeanHr"       "MaxHr"       
[26] "Event"        "Trial"

In [6]:
# column indexes for the response/dependent variables of interest (Comfort and Anxiety)
response_ids = c(3, 4)
responses = colnames(signals)[response_ids]

#---------------------------------------- Models ------------------------------------------#
on_the_fly_max = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ NumPks + MaxPkAmp + MaxEntropy + MaxHr, 
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        data = na.omit(signals),
        method = "ML"
    )
})

on_the_fly_mean = lapply(responses, function(x) {
    lme(
        eval(
            substitute(
                resp ~ NumPks + MeanScl + MeanEntropy + MeanHr, 
                list(resp = as.name(x))
            )
        ),
        random = ~1|pid/Trial,
        data = na.omit(signals),
        method = "ML"
    )
})


In [7]:
# Combine all models
on_the_fly_models = c(self_report_max, self_report_mean)

response_names = c(
    "COMFORT -- MAX-VALUED PREDICTORS",
    "ANXIETY -- MAX-VALUED PREDICTORS",
    "COMFORT -- MEAN-VALUED PREDICTORS",
    "ANXIETY -- MEAN-VALUED PREDICTORS"
)
model_idx  = c(1:4)

In [8]:
cat("ALL MAIN EFFECTS\n")
cat("-------------------------------------------------------------------------------\n")

model_print = function(x, y, z){
    cat(y[x])
    cat("\n")
    print(summary(z[[x]]))
    cat("\n\n\n")
    cat("==============================================================================
    \n\n\n")
}

lapply(model_idx, model_print, y=response_names, z=on_the_fly_models)

ALL MAIN EFFECTS
-------------------------------------------------------------------------------
COMFORT -- MAX-VALUED PREDICTORS
Linear mixed-effects model fit by maximum likelihood
 Data: na.omit(signals) 
       AIC     BIC    logLik
  787.8994 815.299 -385.9497

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:    1.273582

 Formula: ~1 | Trial %in% pid
        (Intercept) Residual
StdDev:   0.4968958 1.125787

Fixed effects: eval(substitute(resp ~ NumPks + MaxPkAmp + MaxEntropy + MaxHr,      list(resp = as.name(x)))) 
                Value Std.Error  DF   t-value p-value
(Intercept)  8.419293 0.7404041 165 11.371213  0.0000
NumPks      -0.193839 0.0676154 165 -2.866786  0.0047
MaxPkAmp    -0.593877 0.2652850 165 -2.238636  0.0265
MaxEntropy  -2.276393 2.1505193 165 -1.058532  0.2914
MaxHr       -0.000546 0.0038628 165 -0.141323  0.8878
 Correlation: 
           (Intr) NumPks MxPkAm MxEntr
NumPks      0.103                     
MaxPkAmp   -0.006 -0.504              
Ma

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

In [14]:
# Read all CSAI-2 signals among others
signals = read.csv("../metrics/all_trails_CSAI2.csv")
colnames(signals)

csai_max <- lme(
    CSAI2 ~ NumPks + MaxHr + MaxPkAmp + MaxEntropy,
    random=~1|pid/Trial,
    data=signals,
    method="ML"
)
csai_mean <- lme(
    CSAI2 ~ NumPks + MeanHr + MeanScl + MeanEntropy,
    random=~1|pid/Trial,
    data=signals,
    method="ML"
)

cat("ALL MAIN EFFECTS\n")
cat("-------------------------------------------------------------------------------\n")
cat("CSAI-2 -- MAX-VALUED PREDICTORS\n")
print(summary(csai_max))
cat("\n\n\n==============================================================================
\n\n\n")
cat("CSAI-2 -- MEAN-VALUED PREDICTORS\n")
print(summary(csai_mean))

[1] "X"            "pid"          "Base_CSAI2"   "CSAI2"        "LatDist"     
 [6] "LongDist"     "MaxVel"       "MeanVel"      "MaxLatAcc"    "MinLatAcc"   
[11] "MeanLatAcc"   "MaxLongAcc"   "MinLongAcc"   "MeanLongAcc"  "MaxLatJerk"  
[16] "MinLatJerk"   "MeanLatJerk"  "MaxLongJerk"  "MinLongJerk"  "MeanLongJerk"
[21] "NumPks"       "MaxPkAmp"     "MeanScl"      "MeanEntropy"  "MaxEntropy"  
[26] "MeanDwell"    "MaxDwell"     "MeanHr"       "MaxHr"        "lf"          
[31] "hf"           "lf_hf"        "Trial"

ALL MAIN EFFECTS
-------------------------------------------------------------------------------
CSAI-2 -- MAX-VALUED PREDICTORS
Linear mixed-effects model fit by maximum likelihood
 Data: signals 
       AIC      BIC    logLik
  375.8485 394.9047 -179.9242

Random effects:
 Formula: ~1 | pid
        (Intercept)
StdDev:    2.251218

 Formula: ~1 | Trial %in% pid
        (Intercept)  Residual
StdDev:    1.620342 0.7550276

Fixed effects: CSAI2 ~ NumPks + MaxHr + MaxPkAmp + MaxEntropy 
                Value Std.Error DF   t-value p-value
(Intercept) 13.223859 2.2566596 56  5.859926  0.0000
NumPks       0.329326 0.0860972 56  3.825051  0.0003
MaxHr       -0.009945 0.0106798 56 -0.931179  0.3558
MaxPkAmp    -0.592062 0.5103020 56 -1.160218  0.2509
MaxEntropy  -2.050975 1.6860624 56 -1.216429  0.2289
 Correlation: 
           (Intr) NumPks MaxHr  MxPkAm
NumPks     -0.233                     
MaxHr      -0.698 -0.031              
MaxPkAmp   -0.118 -0.237  0.041       
MaxEntropy -0.542 -0.0